In [2]:
import numpy as np
from numpy import shape

from datetime import date

import random
import math
import dill
import glob
import gc

# from tensorflow import keras

import matplotlib.pyplot as plt

import tensorflow as tf

from random import seed

# Import train_test_split function
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import device

# Tensorflow soll auf CPU und nicht auf der GPU laufen
device("cpu:0")

# für GPU:
# tf.device("gpu:0")

In [3]:
%run '/home/jovyan/rna/_functions/functions.py'

### Load MNIST images

In [4]:
# # MNIST von sklearn mit 28x28 Pixel
# from sklearn.datasets import fetch_openml

# # Load data from https://www.openml.org/d/554
# X_raw, y_raw = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [5]:
# Import data that was saved on this system

raw_data = load_file(file = "/home/jovyan/rna/MNIST/Raw_Data/MNIST_images.pkl")
X_raw = raw_data[0]
y_raw = raw_data[1]

In [6]:
# Filter on indices of '0' and '8'
indices = (y_raw == '0') | (y_raw == '8')

# Reshape into squares
X_raw = X_raw[indices, :]
X = X_raw.reshape([sum(indices), 28, 28])

y = y_raw[indices]

# Factorize into dichotomous variable
Y = pd.factorize(y)[0]

In [7]:
# We analyse 1998 samples
np.unique(y[0:1998], return_counts=True)

(array(['0', '8'], dtype=object), array([1032,  966]))

### Load Persistences Landscapes

In [8]:
pl_folderpath = "/home/jovyan/rna/MNIST/persistence_landscapes_averages/"

In [9]:
# Eingelesenen Dateien
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

/home/jovyan/rna/MNIST/persistence_landscapes_averages/avgPL_bucket01_H0_scaledWithin_2022-09-06.pkl
/home/jovyan/rna/MNIST/persistence_landscapes_averages/avgPL_bucket01_H1_scaledWithin_2022-09-06.pkl
/home/jovyan/rna/MNIST/persistence_landscapes_averages/avgPL_bucket01_H0_scaledBetween_2022-09-06.pkl
/home/jovyan/rna/MNIST/persistence_landscapes_averages/avgPL_bucket01_H1_scaledBetween_2022-09-06.pkl
/home/jovyan/rna/MNIST/persistence_landscapes_averages/avgPL_bucket01_H0_unscaled_2022-09-06.pkl
/home/jovyan/rna/MNIST/persistence_landscapes_averages/avgPL_bucket01_H1_unscaled_2022-09-06.pkl


In [10]:
avgPL_bucket01_H0_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
avgPL_bucket01_H1_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])

avgPL_bucket01_H0_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
avgPL_bucket01_H1_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])

avgPL_bucket01_H0_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
avgPL_bucket01_H1_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

In [11]:
# Parameter for persistence landscapes
pl_resolution = 250
pl_num_landscapes = 10

### Train-Test-Splits Persistence Landscapes

In [12]:
relevant_samples = len(avgPL_bucket01_H0_scaledBetween)
relevant_samples

1998

In [13]:
# Bucket01 scaledBetween
seed(999)
# Train-Test-Split 
X_b01_scaledBetween_H0_train, X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_test = train_test_split(avgPL_bucket01_H0_scaledBetween,
                                                                                                                                        Y[0:relevant_samples],
                                                                                                                                        test_size=0.2)

seed(999)
# Train-Test-Split 
X_b01_scaledBetween_H1_train, X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_test = train_test_split(avgPL_bucket01_H1_scaledBetween,
                                                                                                                                        Y[0:relevant_samples],
                                                                                                                                        test_size=0.2)

# Bucket01 scaledWithin
seed(999)
# Train-Test-Split 
X_b01_scaledWithin_H0_train, X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_test = train_test_split(avgPL_bucket01_H0_scaledWithin,
                                                                                                                                    Y[0:relevant_samples],
                                                                                                                                    test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_scaledWithin_H1_train, X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_test = train_test_split(avgPL_bucket01_H1_scaledWithin,
                                                                                                                                    Y[0:relevant_samples],
                                                                                                                                    test_size=0.2)

# Bucket01 Unscaled
seed(999)
# Train-Test-Split 
X_b01_unscaled_H0_train, X_b01_unscaled_H0_test, Y_b01_unscaled_H0_train, Y_b01_unscaled_H0_test = train_test_split(avgPL_bucket01_H0_unscaled,
                                                                                                                    Y[0:relevant_samples],
                                                                                                                    test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_unscaled_H1_train, X_b01_unscaled_H1_test, Y_b01_unscaled_H1_train, Y_b01_unscaled_H1_test = train_test_split(avgPL_bucket01_H1_unscaled,
                                                                                                                    Y[0:relevant_samples],
                                                                                                                    test_size=0.2)

# Machine Learning Classification

## Support Vector Machine

In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [15]:
folderpath_results = "/home/jovyan/rna/MNIST/results/"

In [16]:
# Train-test split with data in vector format
image_vector_size = 28*28

X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(X[0:len(avgPL_bucket01_H1_scaledBetween)].reshape(len(avgPL_bucket01_H1_scaledBetween), image_vector_size),
                                                                    np.asarray(y[0:len(avgPL_bucket01_H1_scaledBetween)]),
                                                                    test_size = 0.3,
                                                                    random_state = 2)

##### SVM (raw data)

In [17]:
# Linear - Rawdata
clf = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_raw_train, y_raw_train.ravel())

print("Accuracy SVM on raw data: " + str(clf.score(X_raw_test, y_raw_test))[:10])

evaluation(y_test = y_raw_test,
           y_pred = clf.predict(X_raw_test),
           filename_csv = folderpath_results + "svm_linear_rawdata.csv")

Accuracy SVM on raw data: 0.98166666
Confusion-Matrix:
 [[312   5]
 [  6 277]] 
 
True Negative:  312 
True Positive 277 
False Negative 6 
False Positive 5

TPR: 0.9787985865724381 
TNR:  0.9842271293375394

Accuracy: 0.9816666666666667
F1 (micro): 0.9816666666666667
F1 (macro): 0.981604069402829


In [18]:
# RBF - Rawdata
clf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_raw_train, y_raw_train.ravel())

print("Accuracy SVM on raw data: " + str(clf.score(X_raw_test, y_raw_test))[:10])

evaluation(y_test = y_raw_test,
           y_pred = clf.predict(X_raw_test),
           filename_csv = folderpath_results + "svm_rbf_rawdata.csv")

Accuracy SVM on raw data: 0.98833333
Confusion-Matrix:
 [[310   7]
 [  0 283]] 
 
True Negative:  310 
True Positive 283 
False Negative 0 
False Positive 7

TPR: 1.0 
TNR:  0.9779179810725552

Accuracy: 0.9883333333333333
F1 (micro): 0.9883333333333333
F1 (macro): 0.9883096603956345


##### SVM (Persistent Landscapes - scaledWithin)

In [19]:
# Linear - H0 - scaledWithin
clf_pl_h0 = make_pipeline(StandardScaler(), 
                       SVC(kernel='linear')).fit(X_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_h0.score(X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H0_test,
           y_pred = clf_pl_h0.predict(X_b01_scaledWithin_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.5675
Confusion-Matrix:
 [[132  80]
 [ 93  95]] 
 
True Negative:  132 
True Positive 95 
False Negative 93 
False Positive 80

TPR: 0.5053191489361702 
TNR:  0.6226415094339622

Accuracy: 0.5675
F1 (micro): 0.5675
F1 (macro): 0.563767485548222


In [20]:
# Linear - H1 - scaledWithin
clf_pl = make_pipeline(StandardScaler(), 
                       SVC(kernel='linear')).fit(X_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl.score(X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H1_test,
           y_pred = clf_pl.predict(X_b01_scaledWithin_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.46
Confusion-Matrix:
 [[ 82 121]
 [ 95 102]] 
 
True Negative:  82 
True Positive 102 
False Negative 95 
False Positive 121

TPR: 0.5177664974619289 
TNR:  0.4039408866995074

Accuracy: 0.46
F1 (micro): 0.46
F1 (macro): 0.4586466165413534


In [21]:
# RBF - H0 - scaledWithin
clf_pl_h0 = make_pipeline(StandardScaler(), 
                       SVC(kernel='rbf')).fit(X_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_h0.score(X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H0_test,
           y_pred = clf_pl_h0.predict(X_b01_scaledWithin_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.575
Confusion-Matrix:
 [[171  41]
 [129  59]] 
 
True Negative:  171 
True Positive 59 
False Negative 129 
False Positive 41

TPR: 0.31382978723404253 
TNR:  0.8066037735849056

Accuracy: 0.575
F1 (micro): 0.575
F1 (macro): 0.538845486111111


In [22]:
# RBF - H1 - scaledWithin
clf_pl_rbf = make_pipeline(StandardScaler(), 
                       SVC(kernel='rbf')).fit(X_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_rbf.score(X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_scaledWithin_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_scaledWithin.csv")

Accuracy SVM on peristence landscapes (scaled within sample): 0.475
Confusion-Matrix:
 [[104  99]
 [111  86]] 
 
True Negative:  104 
True Positive 86 
False Negative 111 
False Positive 99

TPR: 0.4365482233502538 
TNR:  0.5123152709359606

Accuracy: 0.475
F1 (micro): 0.47500000000000003
F1 (macro): 0.47393471780355223


##### SVM (Persistent Landscapes - scaledBetween)

In [23]:
# Linear - H0 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H0_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_scaledBetween.csv")

Accuracy SVM : 0.5375
Confusion-Matrix:
 [[112  90]
 [ 95 103]] 
 
True Negative:  112 
True Positive 103 
False Negative 95 
False Positive 90

TPR: 0.5202020202020202 
TNR:  0.5544554455445545

Accuracy: 0.5375
F1 (micro): 0.5375
F1 (macro): 0.5372657407812705


In [24]:
# Linear - H1 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H1_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_scaledBetween.csv")

Accuracy SVM : 0.545
Confusion-Matrix:
 [[114 100]
 [ 82 104]] 
 
True Negative:  114 
True Positive 104 
False Negative 82 
False Positive 100

TPR: 0.5591397849462365 
TNR:  0.5327102803738317

Accuracy: 0.545
F1 (micro): 0.545
F1 (macro): 0.5447154471544715


In [25]:
# RBF - H0 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H0_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_scaledBetween.csv")

Accuracy SVM : 0.5375
Confusion-Matrix:
 [[142  60]
 [125  73]] 
 
True Negative:  142 
True Positive 73 
False Negative 125 
False Positive 60

TPR: 0.3686868686868687 
TNR:  0.7029702970297029

Accuracy: 0.5375
F1 (micro): 0.5375
F1 (macro): 0.5233156616571866


In [26]:
# RBF - H1 - scaledBetween
clf_pl_rbf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_train)

print("Accuracy SVM : " + str(clf_pl_rbf.score(X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_scaledBetween_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_scaledBetween.csv")

Accuracy SVM : 0.54
Confusion-Matrix:
 [[124  90]
 [ 94  92]] 
 
True Negative:  124 
True Positive 92 
False Negative 94 
False Positive 90

TPR: 0.4946236559139785 
TNR:  0.5794392523364486

Accuracy: 0.54
F1 (micro): 0.54
F1 (macro): 0.537037037037037


##### SVM (Persistent Landscapes - unscaled)

In [27]:
# Linear - H0 - unscaled
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_unscaled_H0_train, Y_b01_unscaled_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_unscaled_H0_test, Y_b01_unscaled_H0_test))[:10])

evaluation(y_test = Y_b01_unscaled_H0_test,
           y_pred = clf_pl.predict(X_b01_unscaled_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_unscaled.csv")

Accuracy SVM : 0.5025
Confusion-Matrix:
 [[106  91]
 [108  95]] 
 
True Negative:  106 
True Positive 95 
False Negative 108 
False Positive 91

TPR: 0.46798029556650245 
TNR:  0.5380710659898477

Accuracy: 0.5025
F1 (micro): 0.5025
F1 (macro): 0.5021234808824173


In [28]:
# Linear - H1 - unscaled
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_unscaled_H1_train, Y_b01_unscaled_H1_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_unscaled_H1_test, Y_b01_unscaled_H1_test))[:10])

evaluation(y_test = Y_b01_unscaled_H1_test,
           y_pred = clf_pl.predict(X_b01_unscaled_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_unscaled.csv")

Accuracy SVM : 0.5275
Confusion-Matrix:
 [[104 100]
 [ 89 107]] 
 
True Negative:  104 
True Positive 107 
False Negative 89 
False Positive 100

TPR: 0.5459183673469388 
TNR:  0.5098039215686274

Accuracy: 0.5275
F1 (micro): 0.5275
F1 (macro): 0.5274734203798963


In [29]:
# RBF - H0 - unscaled
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_unscaled_H0_train, Y_b01_unscaled_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_unscaled_H0_test, Y_b01_unscaled_H0_test))[:10])

evaluation(y_test = Y_b01_unscaled_H0_test,
           y_pred = clf_pl.predict(X_b01_unscaled_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_unscaled.csv")

Accuracy SVM : 0.5525
Confusion-Matrix:
 [[153  44]
 [135  68]] 
 
True Negative:  153 
True Positive 68 
False Negative 135 
False Positive 44

TPR: 0.33497536945812806 
TNR:  0.7766497461928934

Accuracy: 0.5525
F1 (micro): 0.5525
F1 (macro): 0.5313369333987891


In [30]:
# RBF - H1 - unscaled
clf_pl_rbf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_unscaled_H1_train, Y_b01_unscaled_H1_train)

print("Accuracy SVM : " + str(clf_pl_rbf.score(X_b01_unscaled_H1_test, Y_b01_unscaled_H1_test))[:10])

evaluation(y_test = Y_b01_unscaled_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_unscaled_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_unscaled.csv")

Accuracy SVM : 0.5375
Confusion-Matrix:
 [[121  83]
 [102  94]] 
 
True Negative:  121 
True Positive 94 
False Negative 102 
False Positive 83

TPR: 0.47959183673469385 
TNR:  0.5931372549019608

Accuracy: 0.5375
F1 (micro): 0.5375
F1 (macro): 0.5353830892001683
